<a href="https://colab.research.google.com/github/hakobjan/Ames-Housing-Data-and-Kaggle-Challenge/blob/master/JIRA_Project_Final_Hakob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
from transformers import pipeline
from nltk import word_tokenize
from nltk.corpus import stopwords
data = pd.read_csv('Defect_Hist_Trends_01_01_22_revised.csv')
df = data.drop_duplicates().copy()

# Preprocessing The Data

In [ ]:
stop_words = set(stopwords.words('english'))
def preprocess(txt):
    txt = str(txt)
    txt = txt.replace('\xa0',' ').strip() # \xa0 adds space
    txt = re.sub(r'http\S+', 'URLREMOVED', txt)
    txt = re.sub(r'\d{1,2}[-\/]\d{1,2}[-\/]\d{4}|\d{4}[-\/]\d{1,2}[-\/]\d{1,2}|\d{1,2}[-\/]\d{1,2}[-\/]\d{1,2}','DATE',txt)
    txt = re.sub(r'\d{2}\:\d{2}\:{0,1}\d{0,2}','TIME', txt)
    txt = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', 'EMAILREMOVED', txt) # remove emails
    txt = re.sub(r'[-_:\/\\,\?\.]',' ', txt) # Replace with a space
    txt = txt.replace('\n',' ').replace('\t',' ').replace('\r',' ') # NOTE: Hope it doeesn't affect txts like url\new
    txt = txt.replace('!',' ').replace('~','').replace('`','')
    txt = re.sub(r'\'t|\'s|’t|’s', '', txt) # remove 's 't
    txt = re.sub(r'[\{\}\[\]\(\)<>\'\"\*”’$;#%^@|=+-]',' ', txt) # Remove
    txt = re.sub(r'[A-Z][a-z]+',lambda x: x.group(0).lower(), txt) # Selective lowercase : https://stackoverflow.com/a/7588961/15505495
    txt = re.sub(r'\d', '', txt) # Remove numbers
    txt = re.sub(r' +',' ',txt) # Remove multiple white spaces
    # Stopword removal
    tokenized = word_tokenize(txt)
    txt = ' '.join([w for w in tokenized if w not in stop_words]) # Stopword removal  
    return txt.strip()

In [ ]:
df['cleaned_summary'] = df.Summary.apply(preprocess) 
df = df.drop(columns=['Summary', 'Description'])

# Selecting The Pipeline & Labels

In [ ]:
classifier = pipeline("zero-shot-classification", device=0)
candidate_labels = ["QA Automation API",
"Uploading Documents",
"Missing Values",
"Anti-virus software",
"DMS",
"New Window Issues",
"Smoke Test",
"NRT",
"Nexus",
"Data",
"Dev Stage Server",
"NWCC",
"WICS",
"Bug",
"Modals",
"DCS",
"Regression Test",
"Validation Test",
"Web Service"
]


No model was supplied, defaulted to facebook/bart-large-mnli (https://huggingface.co/facebook/bart-large-mnli)


Downloading:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [ ]:
%%time
df['class'] = df['cleaned_summary'].apply(lambda x: classifier(str(x), candidate_labels))

/usr/local/lib/python3.7/dist-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,


Generated Classes
CPU times: user 42min 5s, sys: 32.7 s, total: 42min 38s
Wall time: 42min 37s


# Saving Output

In [ ]:
label = []
for i in df['class']:
  label.append(i["labels"][0])

df['Cluster label'] = label
df.to_csv('Jira_output.csv')

Saved Labels to the file
